In [1]:
from pyntcloud import PyntCloud
from pyntcloud.geometry.areas import *

import numpy as np
import pandas as pd

from rotations import *
from sklearn import preprocessing

In [2]:

def is_on_face(s,a,b,c):
    # s is the new point
    # function assumes the face has been transformed to the xy-plane
    as_x = s[0]-a[0]
    as_y = s[1]-a[1]

    s_ab = (b[0]-a[0])*as_y-(b[1]-a[1])*as_x > 0;

    if((c[0]-a[0])*as_y-(c[1]-a[1])*as_x > 0 == s_ab): 
        # is the point s to the left of or to the right of
        # both the lines AB and AC? If true, it can't be inside.
        print("not on face, case 1")
        return False

    if((c[0]-b[0])*(s[1]-b[1])-(c[1]-b[1])*(s[0]-b[0]) > 0 != s_ab): 
        # the point is between the "cones"
        # checking if point is outside the remaining boundary
        print("not on face, case 2")
        return False

    return True;


In [3]:

#diamond = PyntCloud.from_file('cube.off')
diamond = PyntCloud.from_file('bed_0001.off')

v1, v2, v3 = diamond.get_mesh_vertices(False, False)

v1_xyz = v1[:, :3]
v2_xyz = v2[:, :3]
v3_xyz = v3[:, :3]

areas = triangle_area_multi(v1_xyz, v2_xyz, v3_xyz)
probabilities = areas / np.sum(areas)

v1 = v1_xyz[41] #Choosing an arbitrary face to test on
v2 = v2_xyz[41]
v3 = v3_xyz[41]

#choosing a starting point on our face
u = np.random.rand(1, 1)
v = (np.random.rand(1, 1)) * (1-u)

result = pd.DataFrame()

result_xyz = (v1 * u) + (v2 * v) + ((1 - (u + v)) * v3)
starting_point = result_xyz.astype(np.float32)

result["x"] = starting_point[:, 0]
result["y"] = starting_point[:, 1]
result["z"] = starting_point[:, 2]

In [4]:
# Translate face such that starting_point is the origin
translation = starting_point
v1,v2,v3,starting_point = [np.subtract(p, translation) for p in (v1,v2,v3,starting_point)] # make list or dict eventually


# rotate the face to become the xy-plane
z_axis = (0.,0.,1.)
AB = np.subtract(v2[0],v1[0])
AC = np.subtract(v3[0],v1[0])

normal = np.cross(AB, AC)



rot_axis = np.cross(normal, z_axis)
if (np.linalg.norm(rot_axis) != 0):    
    angle = np.arccos(np.dot(normal,z_axis)/(np.linalg.norm(normal)*np.linalg.norm(z_axis)))
    angle = np.rad2deg(angle)
    v1,v2,v3 = [vrotate(p, angle, rot_axis) for p in (v1,v2,v3)] # make list or dict eventually

    print(angle, v1,v2,v3,starting_point)
print(starting_point, v1)

[[0. 0. 0.]] [[-0.17630577 -0.36144638  0.        ]]


In [5]:
r = 0.04

#find angle to rotate by around the normal
theta = np.random.uniform(0, 2*np.pi)
theta = np.rad2deg(theta)
#find the point to rotate around the normal

dist = np.linalg.norm(v1[0])
if dist < r: 
    dist2 = np.linalg.norm(v2)
    dist3 = np.linalg.norm(v3)
    if max(dist2, dist3) < r:
        raise ValueError("r value is too high or the face is too small. Generated points are outside the face")
    dist = max(dist2, dist3)
    
    
rho = np.random.uniform(np.sqrt(r), np.sqrt(2*r))

#print(rho)
parts = (dist/rho) 

pre_rotation_point = ((v1[0][0]) * (1/parts), (v1[0][1]) * (1/parts), 0.)
    
    
print("start", starting_point)
print(pre_rotation_point)
new = vrotate(pre_rotation_point, theta, z_axis)
print("new", new)
print("on face:", is_on_face(new, v1[0], v2[0], v3[0]))

start [[0. 0. 0.]]
(-0.09556253258657216, -0.1959137886105977, 0.0)
new [0.16053396 0.14745595 0.        ]
not on face, case 1
on face: False


In [6]:
print(v1,v2,v3)

[[-0.17630577 -0.36144638  0.        ]] [[0.32369423 1.1385536  0.        ]] [[ 0.32369423 -0.36144638  0.        ]]
